# LAB #2: Numpy

## Introduction
In this laboratory, you will perform some operation with NumPy arrays in such a way to build your first Machine Learning model. 
In particular, you will build a NumPy-based version of the K-Nearest Neighbors algorithm (a.k.a. KNN).

## 0 Preliminary steps
### 0.1 NumPy
Make sure you have the NumPy library installed, its use is strongly recommended for this laboratory.
NumPy is the fundamental package for scientific computing with Python. You can read more about it on
the official documentation.


In [1]:
! pip install numpy

### 0.2 Iris dataset download 
For this lab, you will need two of the datasets you have already met: Iris and MNIST. Please refer to
Laboratory 1 for a complete description of the datasets.
Iris. You can download it from:
https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data

In [2]:
# linux users
# !wget https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data -O iris.csv
# windows users
! pip install wget
import wget
wget.download("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", "iris.csv")

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=7b798ef08d1f3fd3389d9ed57ccb2f10f3c807f8ee3c9997adb328aa19bc280c
  Stored in directory: /home/students/s344024/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


'iris (1).csv'

## 1 Exercises 
Note that exercises marked with a ($\star$) are optional, you should focus on completing the other ones first.

## 1.1 Iris Analysis with Numpy
As you might remember from Lab. 1, the Iris dataset collects the measurements of different Iris flowers,
and each data point is characterized by 4 **features** (sepal length, sepal width, petal length, petal width) and is associated to 1 **label** (i.e. an Iris species - Setosa, Versicolor, or Virginica) which in this case is the last element of the row (last column of the csv file). 

1. Load the Iris dataset. You can use the `csv` library that we saw in the last laboratory or read it with the standard `open(filename, strategy)`. 
In the second case remember to split correctly the different fields, and avoid new line characters. In any case check for empty lines. 
This time remember to store the 4 features in a numpy array `x` of shape (n_sample, 4) and the labels in a different array `y` of shape (n_sample,) converting the 3 different species to a corresponding numerical value. E.g.,
      - Iris-setosa: 0
      - Iris-versicolor: 1
      - Iris-virginica: 2

In order to check you have correctly loaded the data, print the shape of the two arrays: you should find
(150, 4) for `x` and (150,) for `y`.

In [53]:
import pandas as pd
import numpy as np


dataset=pd.read_csv("../iris_new.csv")

dataset_x=dataset.iloc[:,: -1]
dataset_y=dataset.iloc[:,4]

array_x=np.array(dataset_x)
array_y=np.array(dataset_y)

array_y[array_y=="Iris-setosa"]=0
array_y[array_y=="Iris-versicolor"]=1
array_y[array_y=="Iris-virginica"]=2

print(array_x.shape)
array_y.shape



(150, 4)


(150,)

2. Compute again the mean and standard deviation for each class by means of the numpy functions

In [50]:
grouped_mean = dataset.groupby('iris species').mean().reset_index()
grouped_mean


,iris species,sepal length,sepal width,petal lenght,petal width
0,Iris-setosa,5.006,3.418,1.464,0.244
1,Iris-versicolor,5.936,2.770,4.260,1.326
2,Iris-virginica,6.588,2.974,5.552,2.026


In [49]:
grouped_std = dataset.groupby('iris species').std().reset_index()
grouped_std

,iris species,sepal length,sepal width,petal lenght,petal width
0,Iris-setosa,0.352490,0.381024,0.173511,0.107210
1,Iris-versicolor,0.516171,0.313798,0.469911,0.197753
2,Iris-virginica,0.635880,0.322497,0.551895,0.274650


3. Compute the distances among two samples (e.g., the $36^{th}$ and the $81^{th}$, the $13^{th}$ and the $15^{th}$) 
by means of the `np.linalg.norm(a-b)` function which computes the norm of `a-b`, i.e., the euclidean distance between the feature of the `a` and of the `b` samples. 
  - Can you guess if the two couples of samples belong to the same species?
  - From the mean and standard deviations computed before can you guess which species? 

In [43]:
vector36 = array_x[35]
vector81 = array_x[80]
vector13= array_x[12]
vector15 = array_x[14]

distance1 = np.linalg.norm(vector36 - vector81)
distance2 = np.linalg.norm(vector13 - vector15)

print(distance1)
print(distance2)

2.908607914449797
1.4317821063276353


TODO: write your comment here

Based on the features of the tuples inside the dataset, by definition we can say that nearest tuples will have a lower value of euclidean distance.

TODO: write your comment here

In [45]:
print(vector36)
print(vector81)
print(vector13)
print(vector15)

[5.  3.2 1.2 0.2]
[5.5 2.4 3.8 1.1]
[4.8 3.  1.4 0.1]
[5.8 4.  1.2 0.2]


36 = Iris setosa 

81 = Iris versicolor

13 = Iris setosa

15 = Iris setosa

4. Find the k nearest neighbors of a sample in the dataset.
    - Define a function `k_nearest_neighbors(x, x_set, k)` that takes as input a sample `x` and a set of sample (i.e., a matrix) `x_set` and returns the indices of the `k` nearest neighbors of `x` in `x_set`.
        - Reuse the `euclidean_distance` function that you defined before to do so. 
        - Remember that the `x_set` is a matrix of shape ($N_{samples}, N_{features}$), so you have to compute the distance between `x` and each row of `x_set`. 
        - In order to find the indices of the `k` nearest neighbors, you can use the `argsort` function that returns the indices that would sort an array
    - Apply the function to the $36^{th}$ sample of the dataset with $k=5$.
    - Print the indices of the $5$ nearest neighbors.
    - Print the labels of the $5$ nearest neighbors. Can you guess the label of the $36^{th}$ sample?

In [91]:
def k_nearest_neighbors(x, x_set, k):
    dictionary ={}
    for index, element in x_set.iterrows():
        distance = np.linalg.norm(x - element)
        dictionary[index]=distance
    
    valori = sorted(dictionary.values())[1:k+1:]
    indici = []
    
    for value in valori:
        for element in dictionary:
            if value == dictionary[element]:
               indici.append(element)
    return indici
        
indici = k_nearest_neighbors(vector36, dataset_x, 5)
print(indici)
for element in indici:
    print(dataset_y[element])

[49, 1, 2, 40, 28]
Iris-setosa
Iris-setosa
Iris-setosa
Iris-setosa
Iris-setosa


TODO: write your comment here

## 1.2 KNN design and implementation
In this exercise, you will implement your own version of the K-Nearest Neighbors (KNN) algorithm, and you will use it to assign an
Iris species (i.e. a label) to flowers whose species is unknown.

The KNN algorithm is straightforward. Suppose that some measurements (e.g., the iris features) and their
relative label (e.g., the iris species) of a set of samples are known in advance. 

<img src="https://mlarchive.com/wp-content/uploads/2022/09/img2.png" width="800">

Then, whenever we want to label a new sample, we look at the K most similar points (a.k.a. neighbors) and assign a label accordingly. 

<img src="https://mlarchive.com/wp-content/uploads/2022/09/img1-1.png" width="800">


The simplest solution is using a majority voting scheme: if the majority of the neighbors votes for a label, we will go for it. 
This approach is naive only at first sight: the local similarity assumed by KNN happens to be roughly true, as you have seen in the previous exercises.
Even though this reasoning does not generalize well, the KNN provides a valid baseline for your tasks.


1. Let’s identify a portion of our data for which we will try to guess the species. Randomly select 20%
of the records and store the first four columns (i.e. the features representing each flower) into a
two-dimensional numpy array of shape ($N_{test}, 4$), you can call it `X_test` and $N_{test}$ is the 20% of the total number of samples.
For the same records, store the test label column (i.e. the one with the species values) into another array, namely `y_test`. 
This is the data that will be used to test the accuracy of your KNN implementation and its correct functioning (i.e. the testing data).

In [105]:
import numpy as np
np.random.seed(0)

indices = np.arange(150)
np.random.shuffle(indices)
test_len = 20*len(indices) // 100
test_idx = indices[:test_len]

x_test = dataset_x[test_idx]
y_test = dataset_y[test_idx]

print("Number of test samples:", y_test.shape[0])

KeyError: "None of [Int64Index([114,  62,  33, 107,   7, 100,  40,  86,  76,  71, 134,  51,  73,\n             54,  63,  37,  78,  90,  45,  16, 121,  66,  24,   8, 126,  22,\n             44,  97,  93,  26],\n           dtype='int64')] are in the [columns]"

2. Store the remaining 80% of the records in the same way. In this case, use the names X_train andy_train for the arrays.
This is the data that your model will use as ground-truth knowledge (i.e. the training data, from which we extract the knowledge and that we will use for comparison).


In [102]:
from sklearn.model_selection import train_test_split

# Assuming you have your data in variables X and y
X_train, X_test, y_train, y_test = train_test_split(dataset_x, dataset_y, test_size=0.2, random_state=42)

3. Focus now on the KNN technique. 
From the next month, you will use the `scikit-learn` package. Many of its functionalities
are exposed via an object-oriented interface. With this paradigm in mind, implement now the KNN
algorithm and expose it as a Python class. The bare skeleton of your class should look like this (you
are free to add other methods if you want to).

```
class KNearestNeighbors:
    def __init__(self, k):
        """
        Store the value of k in a attribute of the class and initialize other attributes.
        :param k : int, number of neighbors to consider.
        """
        pass # TODO: implement it!
    def fit(self, X, y):
        """
        Store the 'prior knowledge' of you model that will be used
        to predict new labels.
        :param X : input data points, ndarray, shape = (R,C).
        :param y : input labels, ndarray, shape = (R,).
        """
        pass # TODO: implement it!
    
    def predict(self, X):
        """Run the KNN classification on X.
        :param X: input data points, ndarray, shape = (N,C).
        :return: labels : ndarray, shape = (N,).
        """
        pass # TODO: implement it!

```


Implement the `__init__` and `fit` methods first. 
- In the `__init__` method, you should store the value of `k` in a private attribute of the class.
- In the `fit` method you should only store the training data in private attributes of the class.

In [27]:
import numpy as np
import pandas as pd

class KNearestNeighbors:
    
    def __init__(self, k):
        self.k = k
    
    def fit(self, X, y):
        self.X_train = X
        self.Y_train = y
    
    def most_frequent(self, List):
        return max(set(List), key=List.count)
    
    def k_nearest_neighbors(self, x, x_set, k):
        dictionary = {}
        for index, element in x_set.iterrows():
            dictionary[index] = np.linalg.norm(x - element)
        valori = sorted(dictionary.values())[0:k]
        indici = []
        for value in valori:
            for element in dictionary:
                if value == dictionary[element]:
                    indici.append(element)
        return indici
    
    def predict(self, X):
        prediction = {}
        for index, element in X.iterrows():
            indici = self.k_nearest_neighbors(element, self.X_train, self.k)
            lista_classi = [self.Y_train[value] for value in indici]
            prediction[index] = self.most_frequent(lista_classi)
        return prediction



In [ ]:
KNN = KNearestNeighbors(5)

KNN.fit(X_train,y_train)

istanze = pd.DataFrame()

istanze = istanze.append(dataset_x.iloc[0])
istanze = istanze.append(dataset_x.iloc[60])
istanze = istanze.append(dataset_x.iloc[102])
istanze = istanze.append(dataset_x.iloc[149])

classi = KNN.predict(istanze)

print(classi)

4. Implement the `predict` method. The function receives as input a numpy array with N rows and C
columns, corresponding to N flowers. The method assigns to each row one of the three Iris species 
using the KNN algorithm, and returns the predicted species as a numpy array. 

    - For finding nearest neighbours, you can either re-use the previously defined `k_nearest_neighbors` function or 
implement a new one exploiting the numpy broadcasting capabilities in order to avoid iterating over the sample matrix `X`.
    - Then, assign the *predicted label* to each sample using a majority voting scheme, i.e., the label that appears most frequently among the k nearest neighbors. To do so you can use the `np.unique(neighbours_labels, return_count=True)` function that returns the unique labels and their counts. 
    - Finally, return the predicted labels as a numpy array.

5. Now let’s fit the KNN model with the X_train and y_train data. Then, try to use your KNN model
to predict the species for each record in X_test and store them in a nupy array called y_pred.
As we did in the previous lab, check how many Iris species in the array y_pred have been guessed correctly computing with respect to the ones in y_test computing the accuracy. 
    - A prediction is correct if `y_pred[i] == y_test[i]`. To get the accuracy then compute the ratio between the number of correct guesses and the total number of guesses is known. 
    - If all labels are assigned correctly ((y_pred == y_test).all() == True), the accuracy of the model is 100%. 
    - Instead, if none of the guessed species corresponds to the real one ((y_pred == y_test).any() == False), the accuracy is 0%


6. ($\star$) As a software developer, you might want to increase the functionalities of your product and
publish newer versions over time. The better your code is structured and organized, the lower is the
effort to release updates.
As such,  extend your KNN implementation adding the parameter `distance`. This has to be one among:
    - Euclidean distance: $ euclidean(p,q) = \sqrt{\sum_{i=1}^{n} (p_i _- q_i)^2} $
    - Manhattan distance: $ manhattan(p,q) = \sum_{i=1}^n |p_i - q_i|$
    - Cosine distance: $ cosine(p, q) = 1 - \frac{\sum_{i=1}^n p_i q_i}{ \sqrt{\sum^n_{i=1} p^2_i} \cdot \sqrt{\sum^n_{i=1} q_i^2}}$

If any of this distance is not already implemented in `numpy` implement it yourself


7. ($\star$) Again, extend now your KNN implementation by adding the parameter `weights` to the constructor,
as shown below:

```
class KNearestNeighbors:
    def __init__(self, k, distance_metric="euclidean", weights="uniform"):
        self.k = k
        self.distance_metric = distance_metric
        self.weights = weights
```

Change your KNN implementation to accept a new weighting scheme for the labels. If weights=
"distance", weight neighbor votes by the inverse of their distance (for the distance, again, use
distance_metric). The weight for a neighbor of the point p is:

$
w(p, n) = \frac{1}{distance\_metric(p, n)}
$

Instead, if the default is chosen (weights="uniform"), use the majority voting you already implemented
in Exercise 6.

<img src="https://mlarchive.com/wp-content/uploads/2022/09/img5.png">


8. ($\star$) Test the modularity of the implementation applying it on a different dataset. Ideally, you should
not change the code of your KNN python class.
- Download the MNIST dataset and retain only 100 samples per digit. You will end up with a dataset of 1000 samples.
- Define again four numpy arrays as you did in Exercises 2 and 3.
- Apply your KNN as you did for the Iris dataset.
- Evaluate the accuracy on MNIST’s y_test.

In [1]:
# download MNIST dataset

# linux users
#! wget https://raw.githubusercontent.com/dbdmg/data-science-lab/master/datasets/mnist_test.csv -O mnist.csv

# windows users
! pip install wget
import wget
wget.download("https://raw.githubusercontent.com/dbdmg/data-science-lab/master/datasets/mnist_test.csv", "mnist.csv")


  Using cached wget-3.2-py3-none-any.whl


'mnist.csv'

In [ ]:
# extracting MNIST dataset
import pandas as pd
from sklearn.model_selection import train_test_split

dataset_mnist=pd.read_csv("../mnist.csv")
dataset_mnist_y = dataset_mnist["7"]
dataset_mnist_x = dataset_mnist.iloc[:,1:]

x_train, x_test, Y_train, Y_test = train_test_split(dataset_mnist_x, dataset_mnist_y, test_size=0.2, random_state=42)

print("X_train shape:", x_train.shape)
print("X_test shape:", x_test.shape)
print("y_train shape:", Y_train.shape)
print("y_test shape:", Y_test.shape)

In [32]:
# define four numpy arrays x_train, y_train, x_test, y_test
KNN = KNearestNeighbors(5)

KNN.fit(x_train,Y_train)

classi = KNN.predict(x_test)

print(classi)

{5344: 4, 7444: 7, 1731: 9, 8719: 1, 4521: 7, 7453: 5, 576: 3, 7428: 4, 5577: 8, 439: 0, 2750: 6, 5730: 0, 5271: 2, 5652: 0, 3999: 4, 4556: 6, 582: 7, 6289: 3, 7050: 6, 4894: 8, 8697: 7, 2249: 9, 4639: 8, 7191: 2, 5872: 1, 6330: 3, 1963: 9, 8242: 0, 6589: 0, 8846: 5, 321: 9, 2678: 2, 6630: 8, 9885: 6, 9285: 8, 3337: 9, 5459: 1, 251: 9, 3973: 1, 7281: 3, 6773: 6, 35: 7, 8361: 3, 1513: 2, 8579: 6, 39: 1, 4755: 8, 3465: 8, 1760: 5, 2304: 3, 3723: 0, 5538: 6, 4992: 7, 8126: 0, 3032: 6, 7937: 5, 3039: 7, 6824: 7, 2545: 5, 2592: 0, 1188: 1, 5365: 2, 6076: 5, 107: 9, 1315: 1, 4751: 9, 2753: 6, 9752: 0, 8852: 5, 2876: 4, 7073: 2, 799: 8, 3570: 4, 2894: 8, 2927: 1, 7445: 0, 971: 0, 8344: 2, 8574: 1, 2020: 5, 9053: 7, 952: 4, 5358: 3, 3857: 1, 5860: 7, 3145: 3, 3305: 0, 3006: 5, 7436: 5, 8876: 8, 8608: 8, 7379: 9, 9237: 3, 1056: 4, 3154: 4, 3787: 5, 5400: 6, 6452: 3, 7538: 2, 8761: 9, 764: 4, 5018: 3, 4902: 1, 6720: 2, 6842: 0, 2165: 1, 1862: 2, 9974: 3, 8881: 9, 2340: 9, 7712: 8, 5441: 1, 4003:

In [ ]:
# Apply KNN on MNIST
